**DM Compression BC1**

Fonctions usuelles utilisé dans le code et imports : 

In [2]:
from PIL import Image
import numpy as np
from math import log10, sqrt


def load(filename):
    """
    Charge une image à partir d'un fichier et la convertit en une matrice numpy.

    Args:
        filename (str): Le chemin du fichier de l'image à charger.

    Returns:
        ndarray: Matrice représentant l'image chargée.
    """
    toLoad = Image.open(filename)
    return np.array(toLoad)


def psnr(original, compressed):
    """
    Calcule le rapport signal sur bruit (PSNR) entre deux images.

    Args:
        original (ndarray): Matrice représentant l'image originale.
        compressed (ndarray): Matrice représentant l'image compressée.

    Returns:
        float: La valeur PSNR calculée.
    """
    mse = np.mean((original.astype(int) - compressed) ** 2)
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse))
    return psnr


def save(matPix, filename):
    """
    Enregistre une matrice représentant une image dans un fichier.

    Args:
        matPix (ndarray): Matrice représentant l'image à enregistrer.
        filename (str): Le nom du fichier dans lequel enregistrer l'image.
    """
    Image.fromarray(matPix).save(filename)


**Question 1 :**
- fonction padding 
- fonction no padding
- fonction test

In [3]:
#question 1
def padding(matrice):
    """
    Ajoute du remplissage (padding) noir à une image pour garantir que ses dimensions soient des multiples de 4.

    Args:
        matrice (ndarray): Matrice représentant l'image à laquelle ajouter le padding.

    Returns:
        ndarray: Matrice de l'image avec padding.
    """
    # Récupérer les dimensions de l'image
    Hauteur = np.shape(matrice)[1]
    largeur = np.shape(matrice)[0]

    # Calculer le nombre de colonnes de padding nécessaires pour atteindre une dimension multiple de 4
    colonnes_supp_hauteur = (4 - Hauteur % 4) % 4
    colonnes_supp_largeur = (4 - largeur % 4) % 4

    # Créer une nouvelle matrice avec les dimensions ajustées et remplie de pixels noirs
    nouvelleMat = np.zeros((largeur + colonnes_supp_largeur, Hauteur + colonnes_supp_hauteur, 3), dtype=np.uint8)
    nouvelleMat[:largeur, :Hauteur] = matrice

    # Enregistrer l'image avec padding
    save(nouvelleMat, "procnoir.png")

    return nouvelleMat


#verification de la question 1 

def no_padding(matrice, largeur, hauteur):
    """
    Supprime le padding ajouté précédemment à une image, en fonction de ses dimensions d'origine.

    Args:
        matrice (ndarray): Matrice représentant l'image avec padding.
        largeur (int): Largeur originale de l'image avant le padding.
        hauteur (int): Hauteur originale de l'image avant le padding.

    Returns:
        ndarray: Matrice de l'image sans padding.
    """
    # Extraire la partie de l'image correspondant aux dimensions d'origine après le padding
    nouvellemat = matrice[:hauteur, :largeur]

    # Enregistrer l'image sans padding
    save(nouvellemat, "procsansnoir.png")


def verif(image_originale, image_transformee):
    """
    Vérifie si deux images sont identiques.

    Args:
        image_originale (ndarray): Matrice représentant l'image d'origine.
        image_transformee (ndarray): Matrice représentant l'image transformée.

    Returns:
        bool: True si les deux images sont identiques, False sinon.
    """
    return np.array_equal(image_originale, image_transformee)

#section test : 
image_test = load("proc.jpg")#ne pas utiliser celle la directement, pas multiple de 4
padding(image_test) #question 1 

imgae_bord = load("procnoir.png")
no_padding(imgae_bord, image_test.shape[1], image_test.shape[0])

# on compare les deux images 
image_reduite = load("procsansnoir.png")
identique = verif(image_test, image_reduite)

print(f"Image avec padding sauvegardée sous le nom 'procnoir.png'")
print(f"Image réduite sauvegardée sous le nom 'procsansnoir.png'\n")
print(f"Test question 1 fonctionne :  {identique}\n")


Image avec padding sauvegardée sous le nom 'procnoir.png'
Image réduite sauvegardée sous le nom 'procsansnoir.png'

Test question 1 fonctionne :  True



**Question 2 et 3 :** 
- fonction decouper_en_blocs
- fonction reconstruire image (pour verifier que tout c'est bien passé)

In [5]:
#Question 2 
def decouper_en_blocs(matrix):
    """
    Découpe une matrice en blocs 4x4 et les stocke dans une liste.
    
    Args:
        matrix (ndarray): Matrice à découper en blocs 4x4.
        
    Returns:
        list: Liste des blocs 4x4.
    """
    blocs = []  #bloc final
    
    # Récupérer les dimensions de la matrice
    hauteur, largeur, canaux = matrix.shape #canaux pour la couleur, en previsions
    
    # Parcourir la matrice en itérant sur les blocs de taille 4x4
    for i in range(0, hauteur, 4):
        for j in range(0, largeur, 4):
            # Extraire le bloc 4x4 à partir de la matrice
            bloc = matrix[i:i+4, j:j+4, :]
            blocs.append(bloc)  
            
    return blocs  

#Question 3
def reconstruire_image(blocs, img_originale):
    """
    Reconstruit une image à partir d'une liste de blocs 4x4.
    
    Args:
        blocs (list): Liste de blocs 4x4.
        img_originale (ndarray): Image originale à partir de laquelle les blocs ont été extraits.
        
    Returns:
        ndarray: Image reconstruite.
    """

    # Récupérer les dimensions de l'image originale
    hauteur, largeur, canaux = img_originale.shape
    
    # Créer une image vide avec les mêmes dimensions que l'image originale
    image_reconstruite = np.zeros((hauteur, largeur, canaux), dtype=np.uint8)
    
    index = 0  # Initialise l'indice pour accéder aux blocs
    # Parcourir l'image en itérant sur les blocs de taille 4x4
    for i in range(0, hauteur, 4):
        for j in range(0, largeur, 4):
            # Remplacer la section correspondante de l'image reconstruite par le bloc actuel
            image_reconstruite[i:i+4, j:j+4, :] = blocs[index]
            index += 1  # Passer au prochain bloc

    #save l'image reconstruite
    save(image_reconstruite, "imereconstruite.png")
    
    return image_reconstruite  


#section debug : 

#on décope l'image en bloc de 4
imgdecoup = decouper_en_blocs(imgae_bord)

#on la réassemble
imereconstruite = reconstruire_image(imgdecoup,imgae_bord)

print(f"Test question 2 et 3 image de base et image reconstruites sont les mêmes ?  :  {np.array_equal(imereconstruite, imgae_bord)}\n")

Test question 2 et 3 image de base et image reconstruites sont les mêmes ?  :  True



**Question 4 :**
- fonction tronquer 
- fonction générer palette 
- afficher_palette (pure debug)

In [7]:
def tronque(n, p):
    """
    Enlève les p derniers bits de l'entier n.
    """
    # n >> p, Décalage de n de p positions vers la droite
    # équivalent a diviser n par 2^p
    n_apres_decalage = n >> p
    
    # Retourne le résultat du décalage
    return n_apres_decalage

def generer_palette(couleur_a, couleur_b):
    """
    Crée la palette à partir des couleurs a et b.
    
    Args:
        couleur_a (numpy.ndarray): Couleur a.
        couleur_b (numpy.ndarray): Couleur b.
    
    Returns:
        numpy.ndarray: Palette de 4 couleurs.
    """
    # Palette de 4 couleurs avec les canaux rouge, vert et bleu

    palette = np.zeros((4, 3), dtype=np.uint8)
    
    palette[0] = couleur_a
    # Deuxième couleur de la palette : 2/3 de couleur a + 1/3 de couleur b
    palette[1] = np.round(2*couleur_a/3 +couleur_b/3)
    # Troisième couleur de la palette : 1/3 de couleur a + 2/3 de couleur b
    palette[2] = np.round(couleur_a/3 + 2*couleur_b/3)
    palette[3] = couleur_b
    
    return palette

#afficher palette : 
def afficher_palette(palette):
    """
    Affiche la palette comme une image.

    Args:
        palette (numpy.ndarray): Palette à afficher.
    """
    # Créer une image avec le mode RGB et les dimensions de la palette
    img_palette = Image.new("RGB", (palette.shape[0], 1))

    # Convertir la palette en liste de tuples (R, G, B)
    palette_liste = [tuple(couleur) for couleur in palette]

    # Définir les pixels de l'image avec les couleurs de la palette
    img_palette.putdata(palette_liste)

    # Afficher l'image
    img_palette.show()

# section test :

# Test de la fonction tronque
n = 255  # 11111111 en binaire (8 bits)
p = 3    # On veut enlever les 3 derniers bits

print(f"test question 4 tronque on tronc 255 de 3 bit, resultat attendu, 31(11111 en binaire) : {tronque(n, p)}\n")


# Test de la fonction generer_palette
couleur_a = np.array([255, 0, 0])  # Rouge
couleur_b = np.array([0, 0, 255])  # Bleu

print(f"test question 4 palette: \n ")
print("Palette obtenue :\n", generer_palette(couleur_a, couleur_b))
afficher_palette(generer_palette(couleur_a, couleur_b))  #debug



test question 4 tronque on tronc 255 de 3 bit, resultat attendu, 31(11111 en binaire) : 31

test question 4 palette: 
 
Palette obtenue :
 [[255   0   0]
 [170   0  85]
 [ 85   0 170]
 [  0   0 255]]
